## OOP basics
Define a small class with state and behavior.

In [ ]:
class Counter:
    """Minimal counter with increment behavior."""
    def __init__(self, start: int = 0):
        self.value = start

    def increment(self, step: int = 1) -> int:
        self.value += step
        return self.value

c = Counter(5)
print("Start:", c.value)
print("After +1:", c.increment())
print("After +3:", c.increment(3))

## OOP Polymorphism

Create a class called **Animal**. Create two methods **sound** and **sleep**. Create two child classes **Dog** and **Cat** that inherits from the **Animal** class. Override the **sound** method for both the child classes. Create instances of the child classes and demonstrate polymorphism.

In [ ]:
# Simple polymorphism demo
class Animal:
    def sound(self) -> str:  # meant to be overridden
        raise NotImplementedError

    def sleep(self) -> str:
        return "Zzz..."  # shared behavior

class Dog(Animal):
    def sound(self) -> str:
        return "Woof!"

class Cat(Animal):
    def sound(self) -> str:
        return "Meow!"

pets = [Dog(), Cat()]
for pet in pets:
    print(type(pet).__name__, "sound:", pet.sound())
    print(type(pet).__name__, "sleep:", pet.sleep())

## Diamond problem demo
Two parents share the same base. Without cooperative `super()`, one parent gets skipped; with `super()`, both run via the MRO.

In [ ]:
# Diamond problem: compare naive override vs cooperative super()
class Animal:
    def move(self):
        print("Animal moves")

class Walker(Animal):
    def move(self):
        print("Walker moves")

class Flyer(Animal):
    def move(self):
        print("Flyer flies")

class Bat(Walker, Flyer):
    pass

print("Without super():")
Bat().move()

class AnimalCoop:
    def move(self):
        print("AnimalCoop moves")

class WalkerCoop(AnimalCoop):
    def move(self):
        print("WalkerCoop moves")
        super().move()

class FlyerCoop(AnimalCoop):
    def move(self):
        print("FlyerCoop flies")
        super().move()

class BatCoop(WalkerCoop, FlyerCoop):
    def move(self):
        print("BatCoop starts")
        super().move()
        print("BatCoop ends")

print("\nWith cooperative super():")
BatCoop().move()
print("MRO:", [cls.__name__ for cls in BatCoop.mro()])

## Quick note on MRO
Python's Method Resolution Order (MRO) is the order Python follows to look up attributes/methods in multiple inheritance. It is computed using the C3 linearization algorithm and can be inspected with `ClassName.mro()` or `instance.__mro__`. In practice:
- Python searches the current class, then parents from left to right, respecting each parent’s own MRO.
- `super()` walks this order, so cooperative methods should call `super()` to let siblings run.
- Avoid diamond conflicts by making parents compatible (all use `super()` consistently).

## Abstract base classes
Use `abc.ABC` plus `@abstractmethod` to enforce required methods. Attempting to instantiate an abstract class raises `TypeError` until all abstract methods are implemented.

In [ ]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def sound(self) -> str:
        ...

    @abstractmethod
    def sleep(self) -> str:
        ...

class Dog(Animal):
    def sound(self) -> str:
        return "Woof!"

    def sleep(self) -> str:
        return "Dog curls up."

class Cat(Animal):
    def sound(self) -> str:
        return "Meow!"

    def sleep(self) -> str:
        return "Cat naps in the sun."

try:
    Animal()
except TypeError as exc:
    print("Abstract instantiation fails:", exc)

pet = Cat()
print("Pet sound:", pet.sound())
print("Pet sleep:", pet.sleep())

## Simple constructor example
Demonstrates `__init__` setting instance attributes and a helper `__str__` for readable output.

In [ ]:
class Book:
    def __init__(self, title: str, author: str, pages: int):
        self.title = title
        self.author = author
        self.pages = pages

    def __str__(self) -> str:
        return f"'{self.title}' by {self.author} ({self.pages} pages)"

favorite = Book("The Pragmatic Programmer", "Andrew Hunt & David Thomas", 352)
print(favorite)

## Dunder ("__") / Magic methods
These special methods let your objects plug into Python’s protocols (construction, printing, equality, arithmetic, iteration, context managers, etc.). Common ones:
- `__init__` constructor; `__repr__` / `__str__` for readable output.
- `__eq__`, `__lt__`, … for comparisons; `__hash__` when objects are immutable.
- `__len__`, `__iter__`, `__getitem__` for container-like behavior.
- `__enter__` / `__exit__` for `with` blocks; `__call__` to make instances callable.
- Arithmetic hooks: `__add__`, `__sub__`, `__mul__`, `__radd__`, etc. Implement pairs to support both operand orders.
Guidelines: keep them small, return new instances for pure ops, and avoid surprising side effects.

## Operator overloading
Implement dunder methods to make objects work with Python operators. Keep implementations small and return new instances or plain values as appropriate.

In [ ]:
class Vector2D:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Vector2D(x={self.x}, y={self.y})"

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Vector2D):
            return NotImplemented
        return self.x == other.x and self.y == other.y

    def __add__(self, other: "Vector2D") -> "Vector2D":
        return Vector2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other: "Vector2D") -> "Vector2D":
        return Vector2D(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar: float) -> "Vector2D":
        return Vector2D(self.x * scalar, self.y * scalar)

    __rmul__ = __mul__

v1 = Vector2D(2, 3)
v2 = Vector2D(-1, 4)
print("v1 + v2 =", v1 + v2)
print("v1 - v2 =", v1 - v2)
print("2 * v1 =", 2 * v1)